# Blackjack

`通过这个实验，了解Model Free的Monte Carlo的planning和controlling`

所有的实验源代码都在`lib`目录下，来自[dennybritz](https://github.com/dennybritz/reinforcement-learning)，这里只做解读和归总。

## 实验目录

- [Gridworld](https://applenob.github.io/gridworld.html)
- [Blackjack](https://applenob.github.io/black_jack.html)
- [Cliff Walking](https://applenob.github.io/cliff_walking.html)
- [Windy Gridworld](https://applenob.github.io/windy_gridworld.html)


## 本文目录

- [问题介绍](#问题介绍)
- [Policy Evaluation](#Policy-Evaluation)
- [Policy Iteration](#Policy-Iteration)
- [Value Iteration](#Value-Iteration)

## 问题介绍

